1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

        *Наименование вакансии
        *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
        *Ссылку на саму вакансию        
        *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

2)Задание похожее, но подход к сбору данных здесь другой.
Необходимо собрать информацию по продуктам питания с сайтов: https://rskrf.ru/ratings/produkty-pitaniya/ и https://roscontrol.com/category/produkti/ Получившийся список должен содержать:

        *Наименование продукта
        *Категорию продукта        (например Бакалея)
        *Подкатегорию продукта    (апример "Рис круглозерный")
        *Параметр "безопасность"
        *Параметр "качество"
        *Общий балл
        *Сайт, откуда получена информация
Данная структура должна быть одинаковая для продуктов с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

Можно выполнить по своему желанию одно любое задание или оба при желании и возможности.

## Задание 1:

### hh.ru

#### Импорт необходимых библиотек:

In [3]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import time

#### Ввод названия вакансии:

In [720]:
vacancy = input("Введите название вакансии:")
vacancy = re.sub('\+',"%2B",vacancy);
vacancy = re.sub('#',"%23",vacancy);
vacancy = re.sub(' +',"+",vacancy);

Введите название вакансии:программист с++


#### Шаблоны минимальной и максимальной зарплаты:

In [721]:
sal_min_1 = re.compile('от [0-9]+')
sal_min_2 = re.compile('[0-9]+\-')
sal_max_1 = re.compile('до [0-9]+')
sal_max_2 = re.compile('\-[0-9]+')

#### Создание запроса и получение ответа:

In [722]:
main_link = 'https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=' 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
}

In [723]:
vacansy_list = []
salary_list = []
min_salary = []
max_salary = []
url_list = []
source_name_list = []
employer_list = []
pages = 10

In [724]:
for pg_num in range(0,pages):
    # создаем запрос и получаем ответ
    page = str(pg_num);
    html = requests.get(main_link + vacancy + '&page=' + page,headers=headers)
    # проверяем доступность страницы
    if html.status_code is not requests.codes.ok:
        print('Страница ' + html.url + ' не доступна')
        break    
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    # ищем блок с вакансиями
    vacansy_block = bsObj.findAll(class_="bloko-link HH-LinkModifier")
    # создаем список вакансий
    for i,vacansy in enumerate(vacansy_block):
        vacansy_list.append(vacansy_block[i].getText())
    # создаем список информационных элементов по вакансиям
    item_info_block = bsObj.findAll(class_="vacancy-serp-item__row vacancy-serp-item__row_header")
    # создаем список элементов с информациям по зарплатам
    for i,salary in enumerate(item_info_block):
        salary_list.append(item_info_block[i].findChildren()[-1].getText())
        salary_list[i] = re.sub('\xa0',"",salary_list[i])
    # создаем список нижней границы зарплат
    for salary in salary_list:
        if re.findall(sal_min_1,salary):
            min_salary.append(re.findall(sal_min_1,salary))        
        elif re.findall(sal_min_2,salary):
            min_salary.append(re.findall(sal_min_2,salary))
        else:
            min_salary.append('')    
    # создаем список верхней границы зарплат   
    for salary in salary_list:
        if re.findall(sal_max_1,salary):
            max_salary.append(re.findall(sal_max_1,salary))        
        elif re.findall(sal_min_2,salary):
            max_salary.append(re.findall(sal_max_2,salary))
        else:
            max_salary.append('')    
    # создаем список ссылок на вакансию 
    for i,url in enumerate(item_info_block):
        if item_info_block[i].findChildren()[-3].get('href') is not None:
            url_list.append(item_info_block[i].findChildren()[-3].get('href'))    
        else:
            url_list.append(item_info_block[i].findChildren()[-3].findChildren()[0].get('href'))
    # создаем список ссылок на источник данных
    for i,info in enumerate(item_info_block):
        source_name_list.append('hh.ru')
    # ищем блок с работодателями
    employer_block = bsObj.findAll(class_="bloko-link bloko-link_secondary")
    # создаем список работодателей
    for i,employer in enumerate(employer_block):
        employer_list.append(employer_block[i].getText())
    time.sleep(1);

In [725]:
# чистим список нижней границы от ненужных символов    
for i,salary in enumerate(min_salary):
    if salary is not None:
        min_salary[i] = re.sub('от ',"",str(min_salary[i]))
        min_salary[i] = re.sub('-',"",str(min_salary[i]))
        min_salary[i] = min_salary[i][2:-2]
# чистим список верхней границы от ненужных символов           
for i,salary in enumerate(max_salary):
    if salary is not None:
        max_salary[i] = re.sub('до ',"",str(max_salary[i]))
        max_salary[i] = re.sub('-',"",str(max_salary[i]))  
        max_salary[i] = max_salary[i][2:-2]

In [726]:
s_vacansy_list = pd.Series(vacansy_list)
s_min_salary = pd.Series(min_salary)
s_max_salary = pd.Series(max_salary)
s_url_list = pd.Series(url_list)
s_source_name_list = pd.Series(source_name_list)
s_employer_list = pd.Series(employer_list)

In [727]:
df = pd.DataFrame()
df['vacansy'] = s_vacansy_list
df['min_salary'] = s_min_salary
df['max_salary'] = s_max_salary
df['url'] = s_url_list
df['source'] = s_source_name_list
df['employer'] = s_employer_list

In [739]:
df.head(10)

,vacansy,min_salary,max_salary,url,source,employer
0,Программист С++/Qt,70000,,https://ramenskoye.hh.ru/vacancy/31273030?quer...,hh.ru,Cappasity Inc.
1,Программист С++ (графика),80000,,https://ramenskoye.hh.ru/vacancy/32543357?quer...,hh.ru,Cappasity Inc.
2,Программист С++,70000,,https://ramenskoye.hh.ru/vacancy/32323917?quer...,hh.ru,Cappasity Inc.
3,Программист С++,120000,160000,https://ramenskoye.hh.ru/vacancy/33165016?quer...,hh.ru,Комтех-Н
4,Ведущий разработчик / Программист С++ (Compute...,200000,300000,https://ramenskoye.hh.ru/vacancy/33330629?quer...,hh.ru,Photo Lab
5,Программист С++,120000,,https://ramenskoye.hh.ru/vacancy/33260935?quer...,hh.ru,"Прикладная Логистика, АНО НИЦ CALS-Технологий"
6,"Программист С++ (Qt, графика)",80000,170000,https://ramenskoye.hh.ru/vacancy/33471836?quer...,hh.ru,AMS Software
7,Программист С++/Qt,,,https://ramenskoye.hh.ru/vacancy/33258509?quer...,hh.ru,ООО СКАТТ Электроникс
8,Программист С++,115000,210000,https://ramenskoye.hh.ru/vacancy/32948043?quer...,hh.ru,"ЗАО СВЕМЕЛ, Многопрофильное внедренческое пред..."
9,Ведущий программист С++,200000,,https://ramenskoye.hh.ru/vacancy/32773076?quer...,hh.ru,"Эверис, Представительство"


### Superjob

#### Ввод названия вакансии:

In [4]:
vacancy = input("Введите название вакансии:")
vacancy = re.sub('\+',"%2B",vacancy);
vacancy = re.sub('#',"%23",vacancy);
vacancy = re.sub(' +',"%20",vacancy);

Введите название вакансии:программист c#


#### Главный url и user agent:

In [9]:
main_link = 'https://www.superjob.ru/vacancy/search/?keywords=' 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
}

#### Создаем необходимые структуры и шаблоны:

In [10]:
vacansy_list=[]
salary_list=[]
min_salary=[]
max_salary=[]
max_salary=[]
url_list=[]
source_name_list=[]
employer_list=[]
pages = 10

In [11]:
sal_min_1 = re.compile('от[0-9]+₽')
sal_min_2 = re.compile('[0-9]+—')
sal_min_3 = re.compile('[0-9]+₽')
sal_max_1 = re.compile('^[0-9]+₽')
sal_max_2 = re.compile('—[0-9]+₽')
url_re = re.compile('^icMQ_ _1QIBo f-test-link')
employer_re = re.compile('^icMQ_ _205Zx f-test-link')

In [12]:
for pg_num in range(1,pages + 1):
     # создаем запрос и получаем ответ
    page = str(pg_num);
    html = requests.get(main_link + vacancy + '&page=' + page,headers=headers)
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    # создаем блок с вакансиями
    vacansy_block = bsObj.findAll(class_="_3mfro CuJz5 PlM3e _2JVkc _3LJqf")
    if vacansy_block == []:        
        print('На странице '+ main_link + vacancy + '&page=' + page + ' нет вакансий')
        break;
    # создаем список вакансий
    for vacansy in vacansy_block:
        vacansy_list.append(vacansy.getText())
    # создаем блок зарплат
    salary_block = bsObj.findAll(class_="_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz")
    # создаем список зарплат
    for i,salary in enumerate(salary_block):
        salary_list.append(salary.getText()) 
        salary_list[i] = re.sub('\xa0',"",salary_list[i])
    # создаем список нижней границы зарплаты
    for salary in salary_list:
        if re.findall(sal_min_1,salary):
            min_salary.append(re.findall(sal_min_1,salary))        
        elif re.findall(sal_min_2,salary):
            min_salary.append(re.findall(sal_min_2,salary))
        elif re.findall(sal_min_3,salary):
            min_salary.append(re.findall(sal_min_3,salary))
        else:
            min_salary.append('')   
    # создаем список верхней границы зарплаты
    for salary in salary_list:
        if re.findall(sal_max_1,salary):
            max_salary.append(re.findall(sal_max_1,salary))        
        elif re.findall(sal_max_2,salary):
            max_salary.append(re.findall(sal_max_2,salary))
        else:
            max_salary.append('')   
    # создаем блок url
    url_block = bsObj.findAll(class_=url_re)
    # создаем список url
    for i,url in enumerate(url_block):
        url_list.append('https://www.superjob.ru' + url_block[i].get('href'))
    # создаем список ссылок на источник данных
    for i,info in enumerate(vacansy_block):
        source_name_list.append('superjob.ru')
    # создаем блок работодателей
    employer_block = bsObj.findAll(class_=employer_re)
    # создаем список работодателей
    for i,employer in enumerate(employer_block):
        employer_list.append(employer_block[i].getText())
    time.sleep(1);

In [13]:
# чистим список нижней границы от ненужных символов    
for i,salary in enumerate(min_salary):
    if salary is not None:
        min_salary[i] = re.sub('от',"",str(min_salary[i]))
        min_salary[i] = re.sub('—',"",str(min_salary[i]))
        min_salary[i] = re.sub('₽',"",str(min_salary[i]))
        min_salary[i] = min_salary[i][2:-2]
# чистим список верхней границы от ненужных символов    
for i,salary in enumerate(max_salary):
    if salary is not None:
        max_salary[i] = re.sub('—',"",str(max_salary[i]))
        max_salary[i] = re.sub('₽',"",str(max_salary[i]))
        max_salary[i] = max_salary[i][2:-2]

In [14]:
s_vacansy_list2 = pd.Series(vacansy_list)
s_min_salary2 = pd.Series(min_salary)
s_max_salary2 = pd.Series(max_salary)
s_url_list2 = pd.Series(url_list)
s_source_name_list2 = pd.Series(source_name_list)
s_employer_list2 = pd.Series(employer_list)

In [15]:
df2 = pd.DataFrame()
df2['vacansy'] = s_vacansy_list2
df2['min_salary'] = s_min_salary2
df2['max_salary'] = s_max_salary2
df2['url'] = s_url_list2
df2['source'] = s_source_name_list2
df2['employer'] = s_employer_list2

In [16]:
df2.head(10)

,vacansy,min_salary,max_salary,url,source,employer
0,Инженер-программист / Разработчик C#,300000,,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,ФГУ НПП Гамма
1,Инженер-программист C#,,,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,Центральная ППК
2,Программист-разработчик C#,,,https://www.superjob.ru/vakansii/programmist-r...,superjob.ru,АйТи Элемент
3,.NET Developer / Разработчик C#,,,https://www.superjob.ru/vakansii/net-developer...,superjob.ru,Информационные технологии
4,Программист C#,160000,,https://www.superjob.ru/vakansii/programmist-c...,superjob.ru,Сеть медицинских клиник «ИММА» («IMMA»)
5,"Full-Stack developer (Angular, C#, .NET)",150000,190000,https://www.superjob.ru/vakansii/full-stack-de...,superjob.ru,«Группа ГУТА»
6,Senior Web.NET C#/F# разработчик,180000,,https://www.superjob.ru/vakansii/senior-web-32...,superjob.ru,Открытая Клиника
7,Программист C#,110000,,https://www.superjob.ru/vakansii/programmist-c...,superjob.ru,Easy Partners Group
8,Разработчик C#,150000,250000,https://www.superjob.ru/vakansii/razrabotchik-...,superjob.ru,Кадровый Центр на Сокольнической Площади
9,Web-программист / Web-разработчик,150000,200000,https://www.superjob.ru/vakansii/web-programmi...,superjob.ru,Минерал-Инфо


## Задание 2:

In [760]:
main_link = 'https://rskrf.ru/ratings/produkty-pitaniya/' 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
}

In [767]:
products_url_list=[]
subproducts_url_list=[]
product_name_list=[]
category_name_list=[]
subcategory_name_list=[]
safety_list=[]
quality_list=[]
total_score_list=[]
source_list=[]

In [768]:
html = requests.get(main_link)
if html.status_code is not requests.codes.ok:
        print('Страница ' + html.url + ' не доступна')
else:
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    # ищем блок с ссылками на категории продуктов
    products_url_block = bsObj.findAll(class_="category-item")

#### Получаем список ссылок на все категории продуктов:

In [769]:
for i, item in enumerate(products_url_block):
    products_url_list.append('https://rskrf.ru' +  products_url_block[i].find('a').get('href'))

In [770]:
products_url_list

['https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/',
 'https://rskrf.ru/ratings/produkty-pitaniya/kolbasnye-izdeliya/',
 'https://rskrf.ru/ratings/produkty-pitaniya/konditerskie-izdeliya/',
 'https://rskrf.ru/ratings/produkty-pitaniya/konservy/',
 'https://rskrf.ru/ratings/produkty-pitaniya/maslo-rastitelnoe/',
 'https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/',
 'https://rskrf.ru/ratings/produkty-pitaniya/myaso-ptitsa-yaytso/',
 'https://rskrf.ru/ratings/produkty-pitaniya/ovoshchi-frukty/',
 'https://rskrf.ru/ratings/produkty-pitaniya/ryba-i-moreprodukty/',
 'https://rskrf.ru/ratings/produkty-pitaniya/sousy/',
 'https://rskrf.ru/ratings/produkty-pitaniya/khlebobulochnye-izdeliya/']

#### Получаем список ссылок на все подкатегории продуктов:

In [771]:
for url in products_url_list:
    link = url;
    html = requests.get(link)
    if html.status_code is not requests.codes.ok:
        print('Страница ' + html.url + ' не доступна')
        break
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    subproducts_url_block = bsObj.findAll(class_="category-item")
    for i, item in enumerate(subproducts_url_block):
        subproducts_url_list.append('https://rskrf.ru' +  subproducts_url_block[i].find('a').get('href'))

In [774]:
for i,url in subproducts_url_list
    link = url;
    html = requests.get(link)
    if html.status_code is not requests.codes.ok:
        print('Страница ' + html.url + ' не доступна')
        break
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    product_props_url_block = bsObj.findAll(class_="category-item")
    for i, item in enumerate(subproducts_url_block):
        subproducts_url_list.append('https://rskrf.ru' +  subproducts_url_block[i].find('a').get('href'))

'https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/'

In [820]:
link = 'https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/'
html = requests.get(link)
bsObj = bs(html.text,'lxml')
product_names_block = bsObj.find(class_='col-xl-4 col-lg-4 col-md-4 product')
product_names_block.findChild('a')

<a :href="item.url" class="card-features-wrapper">
<div class="card-features" v-if="item.indicator.length &gt; 0">
<div :key="indicator.id" class="feature-item" v-for="indicator in item.indicator">
<span>{{indicatorsList[indicator.id]}}</span>
<div :class="indicator.class" class="word-rating" v-if="indicator.text">
                                        {{indicator.text}}
                                        <span>{{indicator.value}}</span>
</div>
<div class="starrating readonly d-inline-flex flex-row-reverse float-right" v-else="">
<span :class="indicator.class" class="green">{{indicator.value}}</span>
<input :checked="indicator.star === 5" data-name="star[#i#][name]" type="radio" value="5"/><label title="5 star"><svg class="icon icon-star"><use xlink:href="/local/templates/main/sprite/sprite.svg#star"></use></svg></label>
<input :checked="indicator.star === 4" data-name="star[#i#][name]" type="radio" value="4"/><label title="4 star"><svg class="icon icon-star"><use xlink:href="/l

In [821]:
link = 'https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/'
html = requests.get(link)
bsObj = bs(html.text,'lxml')
product_names_block = bsObj.find(class_='col-xl-4 col-lg-4 col-md-4 product')
product_names_block.find(class_='card-body')

<div class="card-body">
<h5 :class="{'with-text': item.s_name }" class="card-title">
<a :href="item.url">
                                    {{item.name}}
                                    <span v-html="item.s_name" v-if="item.s_name"></span>
</a>
</h5>
<div class="card-info">
<div class="starrating readonly d-inline-flex flex-row-reverse float-left">
<span class="green">{{item.points}}</span>
<input :checked="item.star === 5" data-name="star[#i#][name]" type="radio" value="5"/><label title="5 star"><svg class="icon icon-star"><use xlink:href="/local/templates/main/sprite/sprite.svg#star"></use></svg></label>
<input :checked="item.star === 4" data-name="star[#i#][name]" type="radio" value="4"/><label title="4 star"><svg class="icon icon-star"><use xlink:href="/local/templates/main/sprite/sprite.svg#star"></use></svg></label>
<input :checked="item.star === 3" data-name="star[#i#][name]" type="radio" value="3"/><label title="3 star"><svg class="icon icon-star"><use xlink:href="/local/